# Fractional Cascading

## Problem Statement

Given a `k` list of sorted integers and a value `x`. Given a query value `x` return the largest value less than or equal to `x` in each of the `k` lists.

In [1]:
import bisect

In [2]:
arr = [
    [21, 54, 64, 79, 93],
    [27, 35, 46, 47, 72],
    [11, 44, 62, 66, 94],
    [10, 35, 46, 79, 83],
]

X = 60

## `k` binary searches | Slower Queries Less Space

In [3]:
def get_positions_k_bin_search(x): 
    return [bisect.bisect_left(l, x) for l in arr]

In [4]:
get_positions_k_bin_search(X)

[2, 4, 2, 3]

## Unified Binary Search | Faster Queries More Space

In [5]:
# For each of the element in `arr` positions will hold
# the location where the element `arr[i][j]` will be inserted.
positions = []
for i in range(len(arr)):
    positions.append([])
    for j in range(len(arr[i])):
        positions[i].append([[]] * len(arr[i]))
        positions[i][j] = [-1] * len(arr)

In [6]:
for i, l in enumerate(arr):
    for j, e in enumerate(l):
        for k, m in enumerate(arr):
            positions[i][j][k] = int(bisect.bisect_left(m, e))

See where `79` - 4th list 4th element

In [7]:
positions[3][3]

[3, 5, 4, 3]

In [8]:
pos_arr = sorted([
    (y, positions[i][j],)
    for i, x in enumerate(arr)
        for j, y in enumerate(x)
], key=lambda x: x[0])

In [9]:
pos_arr

[(10, [0, 0, 0, 0]),
 (11, [0, 0, 0, 1]),
 (21, [0, 0, 1, 1]),
 (27, [1, 0, 1, 1]),
 (35, [1, 1, 1, 1]),
 (35, [1, 1, 1, 1]),
 (44, [1, 2, 1, 2]),
 (46, [1, 2, 2, 2]),
 (46, [1, 2, 2, 2]),
 (47, [1, 3, 2, 3]),
 (54, [1, 4, 2, 3]),
 (62, [2, 4, 2, 3]),
 (64, [2, 4, 3, 3]),
 (66, [3, 4, 3, 3]),
 (72, [3, 4, 4, 3]),
 (79, [3, 5, 4, 3]),
 (79, [3, 5, 4, 3]),
 (83, [4, 5, 4, 4]),
 (93, [4, 5, 4, 5]),
 (94, [5, 5, 4, 5])]

In [10]:
U = list(zip(*pos_arr))[0]

In [11]:
def get_positions_unified_bin_search(x): 
    index = bisect.bisect_left(U, x)
    if index == len(pos_arr):
        return [len(l) for l in arr]
    return pos_arr[index][1]

In [12]:
get_positions_unified_bin_search(X)

[2, 4, 2, 3]

## Fractional Cascading | Best of both Worlds

In [13]:
MIN_VAL, MAX_VAL = -1000000000, 1000000000

In [14]:
m_arr = []

In [15]:
m_arr.insert(0, [x for x in arr[-1]])

In [16]:
for i in range(len(arr) - 2, -1, -1):
    m_arr.insert(0, sorted([x for k, x in enumerate(m_arr[0]) if k % 2] + arr[i]))

In [17]:
for l in m_arr:
    l.insert(0, MIN_VAL)
    l.append(MAX_VAL)

In [18]:
m_arr

[[-1000000000, 21, 35, 46, 54, 62, 64, 79, 79, 93, 1000000000],
 [-1000000000, 27, 35, 35, 46, 47, 62, 72, 79, 1000000000],
 [-1000000000, 11, 35, 44, 62, 66, 79, 94, 1000000000],
 [-1000000000, 10, 35, 46, 79, 83, 1000000000]]

In [19]:
# For each of the element in `arr` positions will hold
# the location where the element `arr[i][j]` will be inserted.
pointers = []
for i in range(len(m_arr)):
    pointers.append([])
    for j in range(len(m_arr[i])):
        pointers[i].append([[]] * len(arr[i]))
        pointers[i][j] = [-1] * 2

In [20]:
for i, l in enumerate(m_arr):
    for j, m in enumerate(m_arr[i]):
        pointers[i][j] = [
            bisect.bisect_left(arr[i], m_arr[i][j]),
            0 if i == len(m_arr) - 1 else bisect.bisect_left(m_arr[i+1], m_arr[i][j]),
        ]

In [21]:
pointers[:3]

[[[0, 0],
  [0, 1],
  [1, 2],
  [1, 4],
  [1, 6],
  [2, 6],
  [2, 7],
  [3, 8],
  [3, 8],
  [4, 9],
  [5, 9]],
 [[0, 0],
  [0, 2],
  [1, 2],
  [1, 2],
  [2, 4],
  [3, 4],
  [4, 4],
  [4, 6],
  [5, 6],
  [5, 8]],
 [[0, 0], [0, 2], [1, 2], [1, 3], [2, 4], [3, 4], [4, 4], [4, 6], [5, 6]]]

In [22]:
def get_positions_fractional_cascading(x): 
    locations = []
    loc, next_loc = pointers[0][bisect.bisect_left(m_arr[0], x)]
    locations.append(loc)
    for i in range(1, len(m_arr)):
        if x <= m_arr[i][next_loc-1]:
            loc, next_loc = pointers[i][next_loc-1]
        else:
            loc, next_loc = pointers[i][next_loc]
        locations.append(loc)
    return locations

In [23]:
get_positions_fractional_cascading(X)

[2, 4, 2, 3]